In [ ]:
import subprocess as sp
import numpy as np
import matplotlib.pyplot as plt
import skimage
from sklearn.feature_extraction.image import extract_patches_2d


f_width = 1024//2 
f_height = 576//2
f_channels = 1
f_batch_size = 10

command = [ "ffmpeg",
            '-i', '/home/veon/Downloads/Birkebeinertraseen.1024x576p50.3Mb.h264.NRK/Birkebeinertraseen.1024x576p50.3Mb.h264.NRK.mp4',
            '-vf', 'scale='+str(f_width)+':'+str(f_height),
            '-f', 'image2pipe',
            '-pix_fmt', 'gray',
            '-vcodec', 'rawvideo', '-']
pipe = sp.Popen(command, stdout = sp.PIPE, bufsize=10**8)

def readFrames(num) :
    raw_image = pipe.stdout.read(f_batch_size*f_height*f_width*f_channels)
   
    # transform the byte read into a numpy array
    image =  np.fromstring(raw_image, dtype='uint8').astype('float32') / 255
    return image.reshape ((f_batch_size, f_height, f_width, f_channels))


patch_size = (7,7)
rng = np.random.RandomState(0)
batch_size = 500


def getAllPathes(img=None):
    if img is None:
        img = readFrames(1)[0]
    
    r = skimage.util.view_as_windows(img.squeeze(), patch_size, step=patch_size)
    return r.reshape(r.shape[0], r.shape[1],-1)


def getPatches(max_patches):
    for img in readFrames(1):
        data = extract_patches_2d(img, patch_size, max_patches=max_patches,
                                  random_state=rng)

        data = np.reshape(data, (max_patches, -1))
        return data
    
    
def display_img(patches):
    shape = np.shape(patches)
    PATCH_SIDE = patch_size[0]
    im = np.empty([shape[-3]*PATCH_SIDE,shape[-2]*PATCH_SIDE])#, axis=3)
    for yy in range(shape[-3]):
        for xx in range(shape[-2]):
            z = np.reshape(patches[yy][xx], [PATCH_SIDE,PATCH_SIDE])
            im[yy*PATCH_SIDE:(yy+1)*PATCH_SIDE,xx*PATCH_SIDE:(xx+1)*PATCH_SIDE] = z
    
    plt.matshow(im,cmap=plt.cm.gray)
    plt.show()

def _patchGenerator(frames, max_patches):
    while True:
        for img in readFrames(frames):
            data = extract_patches_2d(img, patch_size, max_patches=max_patches,
                                      random_state=rng)

            data = np.reshape(data, (max_patches, -1))
            yield (data, data)
    


In [ ]:
from keras import backend as K
from keras import objectives

from keras.callbacks import History, ModelCheckpoint, LambdaCallback
from IPython import display

def plot_progress(epoch,logs):
    plt.figure()
    plt.plot(range(epoch+1),history.history['loss'],'b',label='trainin loss')
    #plt.plot(range(epoch+1),history.history['val_loss'],'r',label='validation loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.title('training error')
    display.clear_output(wait=True)
    display.display(plt.gcf())
    
history = History()
plot_progress_cb = LambdaCallback(on_epoch_end=plot_progress)

In [ ]:
from keras import regularizers
from keras.layers import Input, Dense, Lambda
from keras.models import Model


import keras.backend.tensorflow_backend as K

with K.tf.device('/gpu:0'):
    K.set_session(K.tf.Session(config=K.tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)))
    

encoding_dim = 2  # use less

samples_per_epoch=5000
nb_epoch=1300
inp_size = patch_size[0]*patch_size[1]

input_img = Input(shape=(inp_size, )) #batch_shape=(batch_size, inp_size)
# add a Dense layer with a L1 activity regularizer
encoded = Dense(encoding_dim, activation='sigmoid',
                activity_regularizer=regularizers.activity_l1(2*10e-6))(input_img)
decoded = Dense(inp_size, activation='sigmoid')(encoded)

autoencoder = Model(input=input_img, output=decoded)

encoder = Model(input=input_img, output=encoded)
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))

autoencoder.compile(optimizer='rmsprop', loss='mse')

import numpy as np
import matplotlib.pyplot as plt

#tb = callbacks.TensorBoard(log_dir='./logs', histogram_freq=2, write_graph=True, write_images=True)

#plot_loss_callback = LambdaCallback(on_epoch_end=lambda epoch, logs: plt.plot(epoch, logs['loss']) )
#plt.plot(np.arange(epoch), logs['loss'])
# print("+ " , epoch, logs['loss'])
#batch_print_callback = callbacks.LambdaCallback(on_epoch_end=lambda batch, logs: print(batch))


#from keras.utils.visualize_util import plot
#plot(autoencoder, to_file='./autoencoder.png')


autoencoder.fit_generator(_patchGenerator(f_batch_size, batch_size), samples_per_epoch, nb_epoch,
                          #shuffle=True,
                         callbacks=[history, plot_progress_cb],
                         verbose=0)


In [ ]:
f = getAllPathes()
x_test = f.reshape(-1, f.shape[-1])

encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

display_img(x_test.reshape(f.shape))
display_img(decoded_imgs.reshape(f.shape))


In [ ]:
x_test = getPatches(10)
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(7, 7))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(7, 7))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

